In [1]:
import numpy as np
import pandas as pd

In [2]:
# https://stackoverflow.com/questions/21971449/how-do-i-increase-the-cell-width-of-the-jupyter-ipython-notebook-in-my-browser

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
import warnings
warnings.simplefilter('ignore')

In [4]:
import os
import time
import copy
import shutil
import cnv_model_utils as u
import cnv_model_classes as c

In [5]:
PATH = "D:/CANCER BIOLOGY/DATASET/TCGA/FROM Xena/"

In [6]:
df_luad = pd.read_csv(PATH+"km_plot_data_meth_luad.csv")
df_lusu = pd.read_csv(PATH+"km_plot_data_meth_lusu.csv")

In [7]:
## drop columns from df_luad
df_luad.drop(columns=['years_to_birth', 'pathologic_stage', 'gender', 'ethnicity'], axis=1, inplace=True)

In [8]:
## drop columns from df_lusu
df_lusu.drop(columns=['years_to_birth', 'pathologic_stage', 'gender', 'ethnicity'], axis=1, inplace=True)

In [9]:
df_luad_data = pd.read_csv(PATH+"meth_luad.csv", index_col=0)
df_lusu_data = pd.read_csv(PATH+"meth_lusu.csv", index_col=0)
df_lusu_data.drop(index=['MBD3L2'], axis=0, inplace=True)

In [10]:
genee = ['C18orf18', 'CCDC15', 'CCNT2', 'EXOC6', 'SNORA9', 'THOP1', 'TNPO2']

In [11]:
df_luad_data = df_luad_data.T
df_lusu_data = df_lusu_data.T

In [12]:
df_final_select_luad = df_luad_data[df_luad_data.columns.intersection(genee)]
df_final_select_lusu = df_lusu_data[df_lusu_data.columns.intersection(genee)]

In [13]:
df_luad.set_index(list(df_luad)[0], inplace=True)
df_lusu.set_index(list(df_lusu)[0], inplace=True)

In [14]:
df_luad

,overall_survival,status
attrib_name,,
TCGA.05.4384,426.0,0.0
TCGA.05.4390,1126.0,0.0
TCGA.05.4396,303.0,1.0
TCGA.05.4405,610.0,0.0
TCGA.05.4410,NaN,NaN
...,...,...
TCGA.NJ.A55O,NaN,NaN
TCGA.NJ.A55R,603.0,0.0
TCGA.NJ.A7XG,617.0,0.0


In [15]:
df_final_select_luad

attrib_name,THOP1,EXOC6,CCNT2,CCDC15,TNPO2,SNORA9,C18orf18
TCGA.05.4384,-0.4807,-0.3753,-0.4673,-0.4783,-0.4582,-0.4635,-0.4711
TCGA.05.4390,-0.4846,-0.3965,-0.4820,-0.4877,-0.4844,-0.4611,-0.4768
TCGA.05.4396,-0.4862,-0.3857,-0.4684,-0.4761,-0.4603,-0.4620,-0.4665
TCGA.05.4405,-0.4892,-0.3956,-0.4779,-0.4843,-0.4860,-0.4567,-0.4667
TCGA.05.4410,-0.4853,-0.3916,-0.4778,-0.4843,-0.4763,-0.4521,-0.4693
...,...,...,...,...,...,...,...
TCGA.NJ.A55O,-0.4705,-0.3911,-0.4391,-0.4659,-0.4612,-0.4638,-0.4487
TCGA.NJ.A55R,-0.4746,-0.3823,-0.4383,-0.4741,-0.4555,-0.4366,-0.4513
TCGA.NJ.A7XG,-0.4841,-0.4122,-0.4865,-0.4797,-0.4826,-0.4643,-0.4791
TCGA.O1.A52J,-0.4776,-0.4105,-0.4626,-0.4680,-0.4666,-0.4297,-0.4531


In [16]:
km_df_luad = pd.concat([df_luad, df_final_select_luad], axis=1, join='inner')
km_df_lusu = pd.concat([df_lusu, df_final_select_lusu], axis=1, join='inner')

In [18]:
km_dataset_meth = pd.concat([km_df_luad, km_df_lusu])

In [19]:
km_dataset_meth

,overall_survival,status,THOP1,EXOC6,CCNT2,CCDC15,TNPO2,SNORA9,C18orf18
TCGA.05.4384,426.0,0.0,-0.4807,-0.3753,-0.4673,-0.4783,-0.4582,-0.4635,-0.4711
TCGA.05.4390,1126.0,0.0,-0.4846,-0.3965,-0.4820,-0.4877,-0.4844,-0.4611,-0.4768
TCGA.05.4396,303.0,1.0,-0.4862,-0.3857,-0.4684,-0.4761,-0.4603,-0.4620,-0.4665
TCGA.05.4405,610.0,0.0,-0.4892,-0.3956,-0.4779,-0.4843,-0.4860,-0.4567,-0.4667
TCGA.05.4410,NaN,NaN,-0.4853,-0.3916,-0.4778,-0.4843,-0.4763,-0.4521,-0.4693
...,...,...,...,...,...,...,...,...,...
TCGA.O2.A52S,387.0,1.0,-0.4845,-0.4883,-0.4628,0.2107,0.4278,-0.4423,-0.4563
TCGA.O2.A52V,1335.0,1.0,-0.4824,-0.4808,-0.4695,0.1433,0.4321,-0.4444,-0.4726
TCGA.O2.A52W,261.0,1.0,-0.4845,-0.4836,-0.4817,0.0449,0.4087,-0.4445,-0.4781
TCGA.O2.A5IB,340.0,1.0,-0.4854,-0.4843,-0.4758,0.2843,0.4106,-0.4448,-0.4752


In [20]:
km_dataset_meth.to_csv(PATH+'curated_km_dataset_meth.csv')

In [ ]:
# ## shuffle the dataset
# df_final = df_final.sample(frac=1, random_state=43)

In [ ]:
## finally, save the dataset 'df_final' to a csv file
## https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html


PATH = "D:/CANCER BIOLOGY/DATASET/TCGA/FROM Xena/"
df_final.to_csv(PATH+'METHYL_DATASET_PREPROCESSED.csv', index=False)

In [ ]:
## check if its being read properly

df_read = pd.read_csv(PATH+'METHYL_DATASET_PREPROCESSED.gz', sep='\t')

In [ ]:
df_read

In [ ]:
from scipy.io.wavfile import read

fs, data = read('alert.wav', mmap=True)  # fs - sampling frequency
data = data.reshape(-1, 1)
import sounddevice as sd
sd.play(data, 44100)

# PREPROCESSING DONE !!